## Data preparation

In [2]:
import pandas as pd
import numpy as np 

type_of_data = "future"
tokens_pair = "btc_busd"
time_interval = "1h"
source = "binance"

load_path = "../data/raw/%s/raw_%s_%s_%s_%s.csv" % (type_of_data, type_of_data, source, tokens_pair, time_interval)
save_path = "../data/original_features/%s/original_%s_%s_%s_%s.csv" % (type_of_data, type_of_data, source, tokens_pair, time_interval)

df = pd.read_csv(load_path)

In [2]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [3]:
df = df[df['open_time'] >= 1513695600000]
df = df[df['open_time'] <= 1655892000000]

In [4]:
df

open_time     open     high      low    close    volume  \
0      1610434800000  36060.5  36615.1  35714.0  36433.4    44.903   
1      1610438400000  36453.3  36496.1  35510.8  35778.9    34.193   
2      1610442000000  35787.8  36130.2  35482.2  35935.4    30.476   
3      1610445600000  35924.4  36105.7  34790.7  35061.2    28.601   
4      1610449200000  35138.1  35766.7  35134.0  35369.2    27.161   
...              ...      ...      ...      ...      ...       ...   
12623  1655877600000  20336.2  20415.9  19924.8  20078.2  6700.248   
12624  1655881200000  20080.9  20228.2  19932.9  20063.3  4767.058   
12625  1655884800000  20063.3  20222.5  20009.6  20199.9  2967.393   
12626  1655888400000  20202.9  20480.0  20200.2  20390.3  4489.490   
12627  1655892000000  20390.4  20526.0  20321.4  20384.2  3454.706   

       quote_asset_volume  number_of_trades  taker_buy_base_asset_volume  \
0            1.613562e+06               342                       28.416   
1            1.230397e+06               368                       18.466   
2            1.093006e+06               330                       12.327   
3            1.008650e+06               315                       21.100   
4            9.621481e+05               302                        6.980   
...                   ...               ...                          ...   
12623        1.348914e+08             53908                     3358.114   
12624        9.584839e+07             40805                     2462.218   
12625        5.976315e+07             27915                     1523.739   
12626        9.123645e+07             40609                     2460.571   
12627        7.047456e+07             31030                     1703.594   

       taker_buy_quote_asset_volume  
0                      1.019288e+06  
1                      6.642936e+05  
2                      4.415400e+05  
3                      7.434806e+05  
4                      2.471991e+05  
...                             ...  
12623                  6.762971e+07  
12624                  4.951619e+07  
12625                  3.069251e+07  
12626                  5.000469e+07  
12627                  3.475732e+07  

[12628 rows x 10 columns]

In [5]:
df.reset_index(drop=True, inplace=True)

## Features engineering with talipp

In [6]:
from talipp.ohlcv import OHLCVFactory
from talipp.indicators import AccuDist, ADX, ALMA, AO, Aroon, ATR, BB, BOP, CCI, ChaikinOsc, ChandeKrollStop, CHOP, CoppockCurve, DEMA, DonchianChannels, DPO, EMA, EMV, ForceIndex, HMA, \
    Ichimoku, KAMA, KeltnerChannels, KST, KVO, MACD, MassIndex, MeanDev, OBV, PivotsHL, ROC, RSI, ParabolicSAR, SFX, SMA, SMMA, SOBV, \
    StdDev, Stoch, StochRSI, SuperTrend, TEMA, TRIX, TSI, UO, VTX, VWAP, VWMA, WMA

price_types = ["open", "close", "high", "low"]
periods = [2,4,8,12,24,48,96,192]

close = df["open"].to_numpy()
high =  df["high"].to_numpy()
low = df["low"].to_numpy()
volume = df["volume"].to_numpy()
open = df["open"].to_numpy()

ohlcv = OHLCVFactory.from_matrix2([
    df["open"].to_numpy(),
    df["high"].to_numpy(),
    df["low"].to_numpy(),
    df["close"].to_numpy(),
    df["volume"].to_numpy()]
)

dct = {
    'close': close,
    'high': high,
    'low': low,
    'open': open,
    'volume': volume
}

def shift(df, col_name):
    df[col_name] = df[col_name].shift(df[col_name].isna().sum())

In [7]:
df["CoppockCurve"] = pd.Series(CoppockCurve(11, 14, 10, close))
df["CoppockCurve"] = df["CoppockCurve"].shift(df["CoppockCurve"].isna().sum())

df["KAMA"] = pd.Series(KAMA(14, 2, 30, close))
df["KAMA"] = df["KAMA"].shift(df["KAMA"].isna().sum())

kst_series = pd.Series(KST(10, 10, 15, 10, 20, 10, 30, 15, 9, close))
df["KST_kst"] = pd.Series(np.vectorize(lambda x: x.kst)(kst_series))
df["KST_signal"] = pd.Series(np.vectorize(lambda x: x.signal)(kst_series))
df["KST_kst"] = df["KST_kst"].shift(df["KST_kst"].isna().sum())
df["KST_signal"] = df["KST_signal"].shift(df["KST_signal"].isna().sum())

macd_series = pd.Series(MACD(12, 26, 9, close))
df["MACD_macd"] = pd.Series(np.vectorize(lambda x: x.macd)(macd_series))
df["MACD_signal"] = pd.Series(np.vectorize(lambda x: x.signal)(macd_series))
df["MACD_macd"] = df["MACD_macd"].shift(df["MACD_macd"].isna().sum())
df["MACD_signal"] = df["MACD_signal"].shift(df["MACD_signal"].isna().sum())

df["MassIndex"] = pd.Series(MassIndex(9, 9, 10, ohlcv))
df["MassIndex"] = df["MassIndex"].shift(df["MassIndex"].isna().sum())

df["OBV"] = pd.Series(OBV(ohlcv))
df["OBV"] = df["OBV"].shift(df["OBV"].isna().sum())

df["KVO"] = pd.Series(KVO(34, 55, ohlcv))
df["KVO"] = df["KVO"].shift(df["KVO"].isna().sum())

df["VWAP"] = pd.Series(VWAP(ohlcv))
df["VWAP"] = df["VWAP"].shift(df["VWAP"].isna().sum())

df["AccuDist"] = pd.Series(AccuDist(ohlcv))
df["AccuDist"] = df["AccuDist"].shift(df["AccuDist"].isna().sum())

df["BOP"] = pd.Series(BOP(ohlcv))
df["BOP"] = df["BOP"].shift(df["BOP"].isna().sum())

df["UO"] = pd.Series(UO(7, 14, 28, ohlcv))
df["UO"] = df["UO"].shift(df["UO"].isna().sum())

df["TSI"] = pd.Series(TSI(13, 25, close))
df["TSI"] = df["TSI"].shift(df["TSI"].isna().sum())

df["AO"] = pd.Series(AO(5, 34, ohlcv))
df["AO"] = df["AO"].shift(df["AO"].isna().sum())

for period in periods:
    df["CCI"+str(period)] = pd.Series(CCI(period, ohlcv))
    shift(df, "CCI"+str(period))    

    df["CHOP"+str(period)] = pd.Series(CHOP(period, ohlcv))
    shift(df, "CHOP"+str(period))    

    vtx_series = pd.Series(VTX(period, ohlcv))
    df["VTX"+str(period)+"_plus_vtx"] = pd.Series(np.vectorize(lambda x: x.plus_vtx)(vtx_series))
    df["VTX"+str(period)+"_minus_vtx"] = pd.Series(np.vectorize(lambda x: x.minus_vtx)(vtx_series))
    df["VTX"+str(period)+"_plus_vtx"] = df["VTX"+str(period)+"_plus_vtx"].shift(df["VTX"+str(period)+"_plus_vtx"].isna().sum())
    df["VTX"+str(period)+"_minus_vtx"] = df["VTX"+str(period)+"_minus_vtx"].shift(df["VTX"+str(period)+"_minus_vtx"].isna().sum()) 

    df["SOBV"+str(period)] = pd.Series(SOBV(period, ohlcv))
    shift(df, "SOBV"+str(period))  
    
    df["ATR"+str(period)] = pd.Series(ATR(period, ohlcv))
    shift(df, "ATR"+str(period))  

    df["ForceIndex"+str(period)] = pd.Series(ForceIndex(period, ohlcv))
    shift(df, "ForceIndex"+str(period)) 
    
    adx_series = pd.Series(ADX(period, period, ohlcv))
    df["ADX_plus_di"] = pd.Series(np.vectorize(lambda x: x.plus_di)(adx_series))
    df["ADX_minus_di"] = pd.Series(np.vectorize(lambda x: x.minus_di)(adx_series))
    df["ADX_adx"] = pd.Series(np.vectorize(lambda x: x.adx)(adx_series))
    df["ADX_plus_di"] = df["ADX_plus_di"].shift(df["ADX_plus_di"].isna().sum())
    df["ADX_minus_di"] = df["ADX_minus_di"].shift(df["ADX_minus_di"].isna().sum())
    df["ADX_adx"] = df["ADX_adx"].shift(df["ADX_adx"].isna().sum())

    
    df["ForceIndex"+str(period)] = pd.Series(ForceIndex(period, ohlcv))
    shift(df, "ForceIndex"+str(period)) 
    
    for price_type in price_types:
        df[price_type+"_max"+str(period)] = df[price_type].rolling(period).max()
        
        df[price_type+"_min"+str(period)] = df[price_type].rolling(period).min()
        
        df[price_type+"_median"+str(period)] = df[price_type].rolling(period).median()
        
        if period >= 12:
            df[price_type+"_quantile25_"+str(period)] = df[price_type].rolling(period).quantile(0.25)
            df[price_type+"_quantile75_"+str(period)] = df[price_type].rolling(period).quantile(0.75)
        
        df[price_type+"_SMA"+str(period)] = pd.Series(SMA(period, dct[price_type]))
        shift(df, price_type+"_SMA"+str(period))

        df[price_type+"_SMMA"+str(period)] = pd.Series(SMMA(period, dct[price_type]))
        shift(df, price_type+"_SMMA"+str(period))

        df[price_type+"_WMA"+str(period)] = pd.Series(WMA(period, dct[price_type]))
        shift(df, price_type+"_WMA"+str(period))

        df[price_type+"_ALMA"+str(period)] = pd.Series(ALMA(period, 0.85, 6, dct[price_type]))
        shift(df, price_type+"_ALMA"+str(period))   

        df[price_type+"_TEMA"+str(period)] = pd.Series(TEMA(period, dct[price_type]))
        shift(df, price_type+"_TEMA"+str(period))   

        df[price_type+"_EMA"+str(period)] = pd.Series(EMA(period, dct[price_type]))
        shift(df, price_type+"_EMA"+str(period)) 

        df[price_type+"_DEMA"+str(period)] = pd.Series(DEMA(period, dct[price_type]))
        shift(df, price_type+"_DEMA"+str(period))  

        df[price_type+"_DPO"+str(period)] = pd.Series(DPO(period, dct[price_type]))
        shift(df, price_type+"_DPO"+str(period)) 

        df[price_type+"_HMA"+str(period)] = pd.Series(HMA(period, dct[price_type]))
        shift(df, price_type+"_HMA"+str(period))

        df[price_type+"_MeanDev"+str(period)] = pd.Series(MeanDev(period, dct[price_type]))
        shift(df, price_type+"_MeanDev"+str(period))  
        
        df[price_type+"_StdDev"+str(period)] = pd.Series(StdDev(period, dct[price_type]))
        shift(df, price_type+"_StdDev"+str(period))  

        df[price_type+"_RSI"+str(period)] = pd.Series(RSI(period, dct[price_type]))
        shift(df, price_type+"_RSI"+str(period))

        df[price_type+"_ROC"+str(period)] = pd.Series(ROC(period, dct[price_type]))
        shift(df, price_type+"_ROC"+str(period))
        
        df[price_type+"_TRIX"+str(period)] = pd.Series(TRIX(period, dct[price_type]))
        shift(df, price_type+"_TRIX"+str(period))

/tmp/ipykernel_9081/1214833122.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["ATR"+str(period)] = pd.Series(ATR(period, ohlcv))
/tmp/ipykernel_9081/1214833122.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["ForceIndex"+str(period)] = pd.Series(ForceIndex(period, ohlcv))
/tmp/ipykernel_9081/1214833122.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1

/tmp/ipykernel_9081/1214833122.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_median"+str(period)] = df[price_type].rolling(period).median()
/tmp/ipykernel_9081/1214833122.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_SMA"+str(period)] = pd.Series(SMA(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

/tmp/ipykernel_9081/1214833122.py:103: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_TEMA"+str(period)] = pd.Series(TEMA(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:106: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_EMA"+str(period)] = pd.Series(EMA(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:109: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

/tmp/ipykernel_9081/1214833122.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_HMA"+str(period)] = pd.Series(HMA(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_MeanDev"+str(period)] = pd.Series(MeanDev(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

/tmp/ipykernel_9081/1214833122.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_HMA"+str(period)] = pd.Series(HMA(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_MeanDev"+str(period)] = pd.Series(MeanDev(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

...

/tmp/ipykernel_9081/1214833122.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_WMA"+str(period)] = pd.Series(WMA(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_ALMA"+str(period)] = pd.Series(ALMA(period, 0.85, 6, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:103: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining

/tmp/ipykernel_9081/1214833122.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_HMA"+str(period)] = pd.Series(HMA(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_MeanDev"+str(period)] = pd.Series(MeanDev(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

/tmp/ipykernel_9081/1214833122.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_ALMA"+str(period)] = pd.Series(ALMA(period, 0.85, 6, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:103: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_TEMA"+str(period)] = pd.Series(TEMA(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:106: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

/tmp/ipykernel_9081/1214833122.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_HMA"+str(period)] = pd.Series(HMA(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_MeanDev"+str(period)] = pd.Series(MeanDev(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

/tmp/ipykernel_9081/1214833122.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_StdDev"+str(period)] = pd.Series(StdDev(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:124: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_RSI"+str(period)] = pd.Series(RSI(period, dct[price_type]))
/tmp/ipykernel_9081/1214833122.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

In [8]:

# print(f'BB: {BB(20, 2, close)[-1]}')
# print(f'ChaikinOsc: {ChaikinOsc(3, 10, ohlcv)[-1]}')
# print(f'ChandeKrollStop: {ChandeKrollStop(10, 2, 9, ohlcv)[-1]}')
# print(f'DonchianChannels: {DonchianChannels(20, ohlcv)[-1]}')
# print(f'Ichimoku: {Ichimoku(26, 9, 52, 52, 26, ohlcv)[-1]}')
# print(f'KeltnerChannels: {KeltnerChannels(20, 26, 1, 1, ohlcv)[-1]}')
# print(f"SAR: {ParabolicSAR(0.02, 0.02, 0.2, ohlcv)[-20:]}")
# print(f'SFX: {SFX(12, 12, 3, ohlcv)[-1]}')
# print(f'Stoch: {Stoch(14, 3, ohlcv)[-1]}')
# print(f'StochRSI: {StochRSI(14, 14, 3, 3, close)[-1]}')
# print(f'SuperTrend: {SuperTrend(10, 3, ohlcv)[-20:]}')


In [9]:
df

open_time     open     high      low    close    volume  \
0      1610434800000  36060.5  36615.1  35714.0  36433.4    44.903   
1      1610438400000  36453.3  36496.1  35510.8  35778.9    34.193   
2      1610442000000  35787.8  36130.2  35482.2  35935.4    30.476   
3      1610445600000  35924.4  36105.7  34790.7  35061.2    28.601   
4      1610449200000  35138.1  35766.7  35134.0  35369.2    27.161   
...              ...      ...      ...      ...      ...       ...   
12623  1655877600000  20336.2  20415.9  19924.8  20078.2  6700.248   
12624  1655881200000  20080.9  20228.2  19932.9  20063.3  4767.058   
12625  1655884800000  20063.3  20222.5  20009.6  20199.9  2967.393   
12626  1655888400000  20202.9  20480.0  20200.2  20390.3  4489.490   
12627  1655892000000  20390.4  20526.0  20321.4  20384.2  3454.706   

       quote_asset_volume  number_of_trades  taker_buy_base_asset_volume  \
0            1.613562e+06               342                       28.416   
1            1.230397e+06               368                       18.466   
2            1.093006e+06               330                       12.327   
3            1.008650e+06               315                       21.100   
4            9.621481e+05               302                        6.980   
...                   ...               ...                          ...   
12623        1.348914e+08             53908                     3358.114   
12624        9.584839e+07             40805                     2462.218   
12625        5.976315e+07             27915                     1523.739   
12626        9.123645e+07             40609                     2460.571   
12627        7.047456e+07             31030                     1703.594   

       taker_buy_quote_asset_volume  ...   low_TEMA192    low_EMA192  \
0                      1.019288e+06  ...           NaN           NaN   
1                      6.642936e+05  ...           NaN           NaN   
2                      4.415400e+05  ...           NaN           NaN   
3                      7.434806e+05  ...           NaN           NaN   
4                      2.471991e+05  ...           NaN           NaN   
...                             ...  ...           ...           ...   
12623                  6.762971e+07  ...  19271.225348  21219.041718   
12624                  4.951619e+07  ...  19275.188612  21205.713824   
12625                  3.069251e+07  ...  19281.635888  21193.318862   
12626                  5.000469e+07  ...  19294.018401  21183.027475   
12627                  3.475732e+07  ...  19310.076334  21174.098693   

        low_DEMA192   low_DPO192    low_HMA192 low_MeanDev192  low_StdDev192  \
0               NaN          NaN           NaN            NaN            NaN   
1               NaN          NaN           NaN            NaN            NaN   
2               NaN          NaN           NaN            NaN            NaN   
3               NaN          NaN           NaN            NaN            NaN   
4               NaN          NaN           NaN            NaN            NaN   
...             ...          ...           ...            ...            ...   
12623  18942.229583   -49.651562  20229.783102     798.409798    1073.338462   
12624  18939.305817 -1430.220312  20252.429588     792.616960    1066.614560   
12625  18938.130432 -1401.197917  20273.158020     784.300130    1055.723955   
12626  18941.024132 -1701.463542  20293.015371     776.114399    1047.184537   
12627  18946.492289 -1499.527604  20312.615217     767.290750    1038.705148   

       low_RSI192  low_ROC192  low_TRIX192  
0             NaN         NaN          NaN  
1             NaN         NaN          NaN  
2             NaN         NaN          NaN  
3             NaN         NaN          NaN  
4             NaN         NaN          NaN  
...           ...         ...          ...  
12623   44.730145  -10.522723   -10.442803  
12624   44.743879   -9.918790   -10.439611  
12625   44.874268  -11.183

In [10]:
df["label"] = (df["close"] - df["close"].shift(1) > 0).shift(-1)

/tmp/ipykernel_9081/945242221.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["label"] = (df["close"] - df["close"].shift(1) > 0).shift(-1)


In [11]:
for c in df.columns:
    print(c)

open_time
open
high
low
close
volume
quote_asset_volume
number_of_trades
taker_buy_base_asset_volume
taker_buy_quote_asset_volume
CoppockCurve
KAMA
KST_kst
KST_signal
MACD_macd
MACD_signal
MassIndex
OBV
KVO
VWAP
AccuDist
BOP
UO
TSI
AO
CCI2
CHOP2
VTX2_plus_vtx
VTX2_minus_vtx
SOBV2
ATR2
ForceIndex2
ADX_plus_di
ADX_minus_di
ADX_adx
open_max2
open_min2
open_median2
open_SMA2
open_SMMA2
open_WMA2
open_ALMA2
open_TEMA2
open_EMA2
open_DEMA2
open_DPO2
open_HMA2
open_MeanDev2
open_StdDev2
open_RSI2
open_ROC2
open_TRIX2
close_max2
close_min2
close_median2
close_SMA2
close_SMMA2
close_WMA2
close_ALMA2
close_TEMA2
close_EMA2
close_DEMA2
close_DPO2
close_HMA2
close_MeanDev2
close_StdDev2
close_RSI2
close_ROC2
close_TRIX2
high_max2
high_min2
high_median2
high_SMA2
high_SMMA2
high_WMA2
high_ALMA2
high_TEMA2
high_EMA2
high_DEMA2
high_DPO2
high_HMA2
high_MeanDev2
high_StdDev2
high_RSI2
high_ROC2
high_TRIX2
low_max2
low_min2
low_median2
low_SMA2
low_SMMA2
low_WMA2
low_ALMA2
low_TEMA2
low_EMA2
low_DEMA2


In [12]:
df.isna().sum().sum()

40607

In [13]:
df.ffill(inplace=True)

In [14]:
df.isna().sum().sum()

40606

In [15]:
df.dropna(inplace=True)

In [16]:
df.reset_index(drop=True, inplace=True)

In [17]:
df

open_time     open     high      low    close    volume  \
0      1612501200000  37316.3  37403.7  37069.9  37164.3    27.436   
1      1612504800000  37192.3  37730.0  37089.6  37484.1    36.881   
2      1612508400000  37494.6  37874.2  37470.6  37758.0    35.075   
3      1612512000000  37797.5  37893.0  37505.0  37797.0    37.867   
4      1612515600000  37836.6  37864.5  37302.3  37342.6    33.577   
...              ...      ...      ...      ...      ...       ...   
12049  1655877600000  20336.2  20415.9  19924.8  20078.2  6700.248   
12050  1655881200000  20080.9  20228.2  19932.9  20063.3  4767.058   
12051  1655884800000  20063.3  20222.5  20009.6  20199.9  2967.393   
12052  1655888400000  20202.9  20480.0  20200.2  20390.3  4489.490   
12053  1655892000000  20390.4  20526.0  20321.4  20384.2  3454.706   

       quote_asset_volume  number_of_trades  taker_buy_base_asset_volume  \
0            1.022543e+06               317                        1.549   
1            1.381300e+06               490                        7.012   
2            1.320395e+06               488                        5.438   
3            1.427648e+06               506                       11.316   
4            1.263689e+06               531                        8.986   
...                   ...               ...                          ...   
12049        1.348914e+08             53908                     3358.114   
12050        9.584839e+07             40805                     2462.218   
12051        5.976315e+07             27915                     1523.739   
12052        9.123645e+07             40609                     2460.571   
12053        7.047456e+07             31030                     1703.594   

       taker_buy_quote_asset_volume  ...    low_EMA192   low_DEMA192  \
0                      5.766583e+04  ...  34897.259134  36141.522905   
1                      2.633490e+05  ...  34919.977693  36173.830670   
2                      2.050385e+05  ...  34946.409012  36213.426114   
3                      4.271747e+05  ...  34972.922908  36253.049440   
4                      3.389543e+05  ...  34997.061531  36287.810985   
...                             ...  ...           ...           ...   
12049                  6.762971e+07  ...  21219.041718  18942.229583   
12050                  4.951619e+07  ...  21205.713824  18939.305817   
12051                  3.069251e+07  ...  21193.318862  18938.130432   
12052                  5.000469e+07  ...  21183.027475  18941.024132   
12053                  3.475732e+07  ...  21174.098693  18946.492289   

        low_DPO192    low_HMA192  low_MeanDev192  low_StdDev192  low_RSI192  \
0     -1013.922917  37378.232970     1526.922808    1785.490544   52.636172   
1     -1029.661979  37414.078744     1529.758767    1782.122487   52.654063   
2     -1240.173958  37450.613004     1532.353223    1777.119610   52.999208   
3     -1135.584375  37487.776734     1535.138900    1771.331633   53.030285   
4     -1004.574479  37524.535417     1535.911361    1761.263347   52.823403   
...            ...           ...             ...            ...         ...   
12049   -49.651562  20229.783102      798.409798    1073.338462   44.730145   
12050 -1430.220312  20252.429588      792.616960    1066.614560   44.743879   
12051 -1401.197917  20273.158020      784.300130    1055.723955   44.874268   
12052 -1701.463542  20293.015371      776.114399    1047.184537   45.197309   
12053 -1499.527604  20312.615217      767.290750    1038.705148   45.401826   

       low_ROC192  low_TRIX192  label  
0       19.211153     0.908082   True  
1       17.692081     0.939370   True  
2       20.582339     0.970752   True  
3       20.856776     1.002225  False  
4       20.903316     1.033707   True  
...           ...          ...    ...  
12049  -10.522723   -10.442803  False  
12050   -9.918790   -10.439611   True  
12051  -11.183314   -10.436076   True  
12052   -9.179930   -10.432119  False  
120

In [18]:
(df["open_time"] - df["open_time"].shift(1) != 3600000).sum()

1

In [19]:
df.to_csv(save_path)